In [2]:
import numpy as np

In [42]:
Horizon = 5
P = np.array([[0.9, 0.1], [0.1, 0.9]])
cost = 0

In [34]:
numStates = 2*Horizon*Horizon
numActions = 2  #(0 refers No Probe and 1 refers to Probe)
gamma = 1
def tuple_to_state (a,b,c):
    return (a*2*Horizon + b*Horizon + c)

# the state tuples are arranged st 
#  state count           tuple 
#        0               (0,0,0)
#        1               (0,0,1)
#        H-1             (0,0,H-1)
#        H               (0,1,0)
#        H+1             (0,1,1)
#        2H-1            (0,1,H-1)
#        2H              (1,0,0)
#        2H+1            (1,0,1)
#        2H^2-1          (1,1,H-1)    

In [35]:
P_exp = np.zeros((Horizon, P.shape[0], P.shape[1]))
for i in range(Horizon):
    if i == 0:
        P_exp[0] = P
    else : 
        P_exp[i] = P_exp[i-1]@P
# print(P_exp[:,:,2])

In [36]:
transitions = np.zeros((numStates,numActions,numStates))
rewards = np.ones((numStates,numActions,numStates)) * -1000

In [43]:
for t in range(Horizon-1):
    for lpr in range(2):
        for lpt in range(t):
            transitions[tuple_to_state(t,lpr,lpt)][0][tuple_to_state(t+1,lpr,lpt)] = 1
            transitions[tuple_to_state(t,lpr,lpt)][1][tuple_to_state(t+1,0,t)] = P_exp[t-lpt][lpr][0]
            transitions[tuple_to_state(t,lpr,lpt)][1][tuple_to_state(t+1,1,t)] = P_exp[t-lpt][lpr][1]
            rewards[tuple_to_state(t,lpr,lpt)][1][tuple_to_state(t+1,0,t)] = -cost
            rewards[tuple_to_state(t,lpr,lpt)][1][tuple_to_state(t+1,1,t)] = -cost
            rewards[tuple_to_state(t,lpr,lpt)][0][tuple_to_state(t+1,lpr,lpt)] = -np.min(P_exp[t-lpt][lpr])


In [44]:
print(np.average(transitions))
print(np.max(rewards))
print((rewards + gamma*np.zeros(numStates)).shape)
print((transitions * (rewards + gamma*np.zeros(numStates))).shape)

0.0048
0.0
(50, 2, 50)
(50, 2, 50)


In [45]:
def vi_solve():
    V = np.zeros(numStates)
    V_old = V
    temp = np.zeros((numStates,numActions))
    while (1):
        temp = np.sum(transitions * (rewards + gamma*V_old), axis=2)
        V = np.max(temp, axis=1)
        if np.allclose(V, V_old,0,1e-10):
            break
        V_old = V
    policy = np.argmax(temp, axis=1)
    return V, policy

In [46]:
V, policy = vi_solve()
print(V, policy)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.] [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 1 1 0 0 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0]


In [48]:
print(np.sum(policy))

12


In [ ]:
def find_value(mdp, Policy_fn):
    T_pi = mdp.transitions[np.arange(mdp.numStates), Policy_fn]
    R_pi = mdp.rewards[np.arange(mdp.numStates), np.array(Policy_fn)]
    V = np.squeeze(np.matmul(np.linalg.inv(np.eye(mdp.numStates) - mdp.gamma * T_pi),np.sum(T_pi * R_pi, axis=1)))
    return V

def hpi_solve(mdp):
    pi = np.random.randint(low=0, high=mdp.numActions, size=mdp.numStates)
    pi_old = pi
    while (1):
        V = find_value(mdp, pi_old)
        pi = np.argmax(np.sum(mdp.transitions * (mdp.rewards + mdp.gamma*V), axis=2), axis=1)
        if np.array_equal(pi, pi_old):
            break
        pi_old = pi
    return V, pi